In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

In [ ]:
def sigmoid(z):
    s = 1 / (1  + np.exp(-z))    
    return s

In [ ]:
def initialize_with_zeros(dim):
    w = np.zeros((dim, 1))
    b = 0.0    
    return w, b

In [ ]:
def propagate(w, b, X, Y):
    
    m = X.shape[1] 
    
    # forward propagation
    A = sigmoid(np.dot(w.T, X) + b) 
    cost = (-1/m) * np.sum(Y*np.log(A) + (1-Y)*np.log(1-A))
    
    # backward propagation
    dw = (1/m) * np.dot(X, (A-Y).T) # (p, m) * (m, 1)
    db = (1/m) * np.sum(A-Y)

    # # verfication
    # assert(dw.shape == w.shape)
    # assert(db.dtype == float)
    
    cost = np.squeeze(np.array(cost))
    
    grads = {"dw": dw, 
             "db": db}
    
    return grads, cost

In [ ]:
def optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False):    
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    
    costs = []
    
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update params
        w -= learning_rate * dw
        b -= learning_rate * db
        
        # record the costs
        if i % 100 == 0:
            costs.append(cost)
            if print_cost:
                print ("Cost after iteration %i: %f" %(i, cost))
                
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [ ]:
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # compute vector "A" predicting the probabilities 
    A = sigmoid(np.dot(w.T,X) + b) # (1, m)
    Y_prediction = (A >= 0.5) * 1.0
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

## All Together


- Initialize (w,b)
- Optimize the loss iteratively to learn parameters (w,b):
    - Computing the cost and its gradient
    - Updating the parameters using gradient descent
- Use the learned (w,b) to predict the labels for a given set of examples

In [ ]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    w, b = initialize_with_zeros(X_train.shape[0])
    
    params, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)

    w = params["w"]
    b = params["b"]
    
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    
    # Print train/test Errors
    if print_cost:
        print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
        print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d